In [1]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
import torch
import os
import numpy as np
import faiss
import pandas as pd
import matplotlib.pyplot as plt
import time
import torchaudio
import gc
import math
bundle= torchaudio.pipelines.HUBERT_BASE
model= bundle.get_model()
directory= "/Users/ariel/Desktop/Hubert-PING/cats_dogs/"


/Users/ariel/anaconda3/envs/faiss/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/0d/vjcj95td2j94d9kd7fq31ylw0000gn/T/ipykernel_29281/2638890144.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import numpy as np


# Définition de la fonction exponentielle négative
def exp_negative(x):
    return math.exp(-x)

#j'ai du ajouter un traitement pour les vecteurs 1D car le vecteur de requete contient une seul vecteur et ne peut etre en 2D car comme tu le sais notre processus de traitement des audios nous donne une longue liste contenant tous les caractéritiques dont on a besoin, voila pourquoi le vecteur de requeue est en 1 dimension j'ai donc améliorer la logique de mon code 
def normalization(embeddings):
    # Convertir en un tableau NumPy si ce n'est pas déjà le cas
    #embeddings = np.array(embeddings)

    # Vérifie si embeddings est un vecteur seul (1D) ou une matrice (2D)
    if embeddings.ndim == 1:
        # Normaliser un vecteur seul
        norm = np.linalg.norm(embeddings)
        if norm == 0:
            return embeddings
        return embeddings / norm
    else:
        # Normaliser chaque ligne d'une matrice
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        return embeddings / norms
    
def get_audio_embedding(audio_path):
    waveform1, sample_rate1=torchaudio.load(audio_path)
    waveform1=torchaudio.functional.resample(waveform1, sample_rate1, bundle.sample_rate)
    with torch.inference_mode():
            emission1, _ = model(waveform1)

    # Aplatir les deux premières dimensions et garder la troisième
    flattened_features1 = emission1.view(-1, emission1.size(2))
    mean_features1 = flattened_features1.mean(dim=0)
    mean1_array=mean_features1.cpu().numpy().astype(np.float32) 
    mean1_normal=normalization(mean1_array)
    mean1_normal_2d = mean1_normal[np.newaxis, :]
    return mean1_normal_2d


def searchinIndex(index, normal_embedding):
    D,I= index.search(normal_embedding, index.ntotal)
    r=pd.DataFrame({'distance':D[0],'index':I[0]})
    return r

In [3]:
index = faiss.read_index("/Users/ariel/Downloads/animals.index")

In [4]:
chemin_noms_animaux = '/Users/ariel/Downloads/noms_animaux.txt'

# Lire le contenu du fichier et le convertir en liste
with open(chemin_noms_animaux, 'r') as fichier:
    # Utiliser une compréhension de liste pour traiter chaque ligne
    names = [line.strip().strip("'").strip(",").strip() for line in fichier.readlines()]



In [5]:
query_audio=get_audio_embedding("/Users/ariel/Downloads/69.wav")

In [6]:
index.ntotal

4430

In [7]:
len(names)

4430

In [8]:

i=searchinIndex(index,query_audio)

In [9]:
print(i)

      distance  index
0     0.113787   3973
1     0.119405   3997
2     0.119617   3974
3     0.120283   4088
4     0.132277   3927
...        ...    ...
4425  1.686773   1376
4426  1.688594   3540
4427  1.694098   2809
4428  1.699397   3442
4429  1.709439   1344

[4430 rows x 2 columns]


In [10]:
i['percentage'] = i['distance'].apply(exp_negative)*100

In [11]:
i

,distance,index,percentage
0,0.113787,3973,89.244822
1,0.119405,3997,88.744811
2,0.119617,3974,88.726005
3,0.120283,4088,88.666916
4,0.132277,3927,87.609804
...,...,...,...
4425,1.686773,1376,18.511601
4426,1.688594,3540,18.477906
4427,1.694098,2809,18.376485
4428,1.699397,3442,18.279370


In [12]:
def get_name_from_index(index):
    return names[index]

i['names'] = i['index'].apply(get_name_from_index)

In [13]:
i

,distance,index,percentage,names
0,0.113787,3973,89.244822,cat_605.wav'
1,0.119405,3997,88.744811,cat_455.wav'
2,0.119617,3974,88.726005,cat_577.wav'
3,0.120283,4088,88.666916,cat_315.wav'
4,0.132277,3927,87.609804,cat_257.wav'
...,...,...,...,...
4425,1.686773,1376,18.511601,dogs_333_stretching.wav'
4426,1.688594,3540,18.477906,dogs_394.wav'
4427,1.694098,2809,18.376485,dogs_394_stretching.wav'
4428,1.699397,3442,18.279370,dogs_521.wav'


In [14]:
def name_normalisation(name):
    if 'dog' in name:
        return "Chien"
    elif 'cat' in name:
        return "Chat"
    elif 'bird' in name:
        return "Oiseau"
    else:
        return "Animal non reconnu"



In [15]:
i['names_normalized'] = i['names'].apply(name_normalisation)

In [16]:
i

,distance,index,percentage,names,names_normalized
0,0.113787,3973,89.244822,cat_605.wav',Chat
1,0.119405,3997,88.744811,cat_455.wav',Chat
2,0.119617,3974,88.726005,cat_577.wav',Chat
3,0.120283,4088,88.666916,cat_315.wav',Chat
4,0.132277,3927,87.609804,cat_257.wav',Chat
...,...,...,...,...,...
4425,1.686773,1376,18.511601,dogs_333_stretching.wav',Chien
4426,1.688594,3540,18.477906,dogs_394.wav',Chien
4427,1.694098,2809,18.376485,dogs_394_stretching.wav',Chien
4428,1.699397,3442,18.279370,dogs_521.wav',Chien


In [17]:
def bayes_theorem(df, n_top_vectors=50):
    # Limite le DataFrame aux n_top_vectors premiers vecteurs
    df_limited = df.head(n_top_vectors)
    # Récupère les catégories uniques et initialise le dictionnaire des probabilités a posteriori
    categories = df_limited['names_normalized'].unique()
    probas_a_posteriori = {categorie: 0 for categorie in categories}
    # Calcul des probabilités a priori uniformes
    probas_a_priori = 1/3
    # Somme des similarités pour chaque catégorie limitée aux 50 premiers vecteurs
    for categorie in categories:
        somme_similarites = df_limited[df_limited['names_normalized'] == categorie]['percentage'].sum()
        probas_a_posteriori[categorie] = somme_similarites * probas_a_priori
    # Normalisation des probabilités a posteriori
    total_proba = sum(probas_a_posteriori.values())
    probas_a_posteriori_normalisees = {categorie: (proba / total_proba) for categorie, proba in probas_a_posteriori.items()}
    return probas_a_posteriori_normalisees
 
# Supposons que 'df' est votre DataFrame
# df = pd.read_csv('chemin_vers_votre_fichier.csv')  # Exemple de chargement de données
 
# Utilisez la fonction sur les 50 premiers vecteurs de votre DataFrame
resultat = bayes_theorem(i, 50)
print(resultat)

{'Chat': 0.8196493915515649, 'Chien': 0.16083023616373518, 'Oiseau': 0.019520372284699904}
